In [ ]:
import numpy as np

def getData(Numbers):
    file = open("res/zip.train", "r") 
    Data=[]    
    for line in file:
        for number in Numbers:
            tmpLine = line.split()
            if(float(tmpLine[0]) == float(number)):
                floatLine = []
                Linelen = len(tmpLine)            
                for i in range(0,Linelen):
                    floatLine.append(float(tmpLine[i]))
                Data.append(floatLine)                
    file.closed   
    return Data
